In [1]:
import pandas as pd 
import numpy as np

In [2]:
df = pd.read_csv('OneDrive\Documents\match_data7-15.csv')
df

,Map,Map Duration,Team1,Team2,Team1 Defense,Team2 Defense,Team1 Attack,Team2 Attack,Team1 OT,Team2 OT,...,Map Code,Total Rounds,Team1 Score,Team2 Score,Team1 rwp,Team2 rwp,Team1 Attack Percent,Team2 Attack Percent,Team1 Defense Percent,Team2 Defense Percent
0,Breeze,55:08,NRG Esports,FURIA,8,6,5,4,0,0,...,6,23,13,10,0.565217,0.434783,0.384615,0.400000,0.615385,0.600000
1,Ascent,48:14,NRG Esports,FURIA,7,6,6,2,0,0,...,5,21,13,8,0.619048,0.380952,0.461538,0.250000,0.538462,0.750000
2,Bind,1:06:54,Cloud9,MIBR,5,4,8,7,0,0,...,3,24,13,11,0.541667,0.458333,0.615385,0.636364,0.384615,0.363636
3,Split,47:29,Cloud9,MIBR,6,7,2,6,0,0,...,0,21,8,13,0.380952,0.619048,0.250000,0.461538,0.750000,0.538462
4,Lotus,57:07,Cloud9,MIBR,8,7,5,3,0,0,...,4,23,13,10,0.565217,0.434783,0.384615,0.300000,0.615385,0.700000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198,Ascent,41:31,LOUD,100 Thieves,0,11,1,2,0,0,...,5,14,1,13,0.071429,0.928571,1.000000,0.153846,0.000000,0.846154
199,Bind,43:12,LOUD,100 Thieves,7,6,6,2,0,0,...,3,21,13,8,0.619048,0.380952,0.461538,0.250000,0.538462,0.750000
200,Lotus,52:22,LOUD,100 Thieves,7,5,6,5,0,0,...,4,23,13,10,0.565217,0.434783,0.461538,0.500000,0.538462,0.500000
201,Bind,1:31:34,Sentinels,G2 Esports,0,6,6,7,0,0,...,3,19,6,13,0.315789,0.684211,1.000000,0.538462,0.000000,0.461538


In [3]:
#creat ML model 
from sklearn.ensemble import RandomForestClassifier

In [36]:
rf = RandomForestClassifier(n_estimators =100, min_samples_split=10, random_state=2) 

In [37]:
predictors = ["Team 1 Code", "Team 2 Code", "Map Code"] 

In [38]:
training = df[df.index <= 163]
validation = df[df.index > 163]

In [39]:
rf.fit(training[predictors], training['Winner'])

RandomForestClassifier(min_samples_split=10, random_state=2)

In [40]:
preds = rf.predict(validation[predictors])

In [41]:
from sklearn.metrics import accuracy_score

In [42]:
acc = accuracy_score(validation["Winner"],preds)
acc

0.6153846153846154

In [55]:
combined = pd.DataFrame(dict(actual=validation["Winner"], predicted=preds))

In [56]:
pd.crosstab(index=combined["actual"], columns=combined["predicted"])

predicted,0,1
actual,,
0,13,1
1,14,11


In [53]:
from sklearn.metrics import precision_score

precision_score(validation["Winner"],preds)

0.9166666666666666

In [74]:
#trying a different model 
from sklearn.svm import LinearSVC 

#random seed 
np.random.seed(45)

clf = LinearSVC(max_iter=100000) 
clf.fit(training[predictors], training['Winner'])

clf.score(validation[predictors], validation['Winner'])

0.46153846153846156

In [44]:
team_mapping = {"Sentinels": 0, "Cloud9": 1, "G2 Esports": 2, "KRÜ Esports": 3, "NRG Esports": 4, "Leviatán": 5, "LOUD": 6, "100 Thieves": 7, "MIBR":8,"FURIA":9,"Evil Geniuses":10}

In [45]:
team_groupings = {}
for team in team_mapping.keys():
    group = df[(df['Team1'] == team) | (df['Team2'] == team)]
    team_groupings[team] = group

In [46]:
grouped_matches1 = df.groupby("Team1") 
grouped_matches1.get_group("MIBR")

,Map,Map Duration,Team1,Team2,Team1 Defense,Team2 Defense,Team1 Attack,Team2 Attack,Team1 OT,Team2 OT,...,Team1 rwp,Team2 rwp,Team1 Attack Percent,Team2 Attack Percent,Team1 Defense Percent,Team2 Defense Percent,Team2 rwp_rolling,Team2 Attack Percent_rolling,Team2 Defense Percent_rolling,Team2 Score_rolling
32,Split,1:01:27,MIBR,Sentinels,5,10,2,3,0,0,...,0.350000,0.650000,0.285714,0.230769,0.714286,0.769231,NaN,NaN,NaN,NaN
33,Ascent,45:03,MIBR,Sentinels,1,7,5,6,0,0,...,0.315789,0.684211,0.833333,0.461538,0.166667,0.538462,NaN,NaN,NaN,NaN
144,Ascent,1:05:24,MIBR,Leviatán,3,6,6,7,0,0,...,0.409091,0.590909,0.666667,0.538462,0.333333,0.461538,NaN,NaN,NaN,NaN
145,Icebox,45:04,MIBR,Leviatán,4,9,3,4,0,0,...,0.350000,0.650000,0.428571,0.307692,0.571429,0.692308,NaN,NaN,NaN,NaN
174,Icebox,48:49,MIBR,100 Thieves,2,9,3,4,0,0,...,0.277778,0.722222,0.600000,0.307692,0.400000,0.692308,NaN,NaN,NaN,NaN
175,Split,42:19,MIBR,100 Thieves,4,10,2,3,0,0,...,0.315789,0.684211,0.333333,0.230769,0.666667,0.769231,NaN,NaN,NaN,NaN
184,Sunset,1:02:30,MIBR,FURIA,4,6,6,7,0,0,...,0.434783,0.565217,0.600000,0.538462,0.400000,0.461538,NaN,NaN,NaN,NaN
185,Icebox,43:02,MIBR,FURIA,4,9,3,4,0,0,...,0.350000,0.650000,0.428571,0.307692,0.571429,0.692308,NaN,NaN,NaN,NaN


In [47]:
grouped_matches2 = df.groupby("Team2") 
grouped_matches2.get_group("MIBR")

,Map,Map Duration,Team1,Team2,Team1 Defense,Team2 Defense,Team1 Attack,Team2 Attack,Team1 OT,Team2 OT,...,Team1 rwp,Team2 rwp,Team1 Attack Percent,Team2 Attack Percent,Team1 Defense Percent,Team2 Defense Percent,Team2 rwp_rolling,Team2 Attack Percent_rolling,Team2 Defense Percent_rolling,Team2 Score_rolling
2,Bind,1:06:54,Cloud9,MIBR,5,4,8,7,0,0,...,0.541667,0.458333,0.615385,0.636364,0.384615,0.363636,NaN,NaN,NaN,NaN
3,Split,47:29,Cloud9,MIBR,6,7,2,6,0,0,...,0.380952,0.619048,0.250000,0.461538,0.750000,0.538462,NaN,NaN,NaN,NaN
4,Lotus,57:07,Cloud9,MIBR,8,7,5,3,0,0,...,0.565217,0.434783,0.384615,0.300000,0.615385,0.700000,NaN,NaN,NaN,NaN
13,Split,1:14:12,FURIA,MIBR,2,5,7,8,0,0,...,0.409091,0.590909,0.777778,0.615385,0.222222,0.384615,NaN,NaN,NaN,NaN
14,Breeze,56:26,FURIA,MIBR,8,9,3,4,0,0,...,0.458333,0.541667,0.272727,0.307692,0.727273,0.692308,NaN,NaN,NaN,NaN
25,Bind,1:16:03,Cloud9,MIBR,5,5,7,7,0,2,...,0.461538,0.538462,0.583333,0.500000,0.416667,0.357143,NaN,NaN,NaN,NaN
26,Ascent,56:21,Cloud9,MIBR,5,6,4,7,0,0,...,0.409091,0.590909,0.444444,0.538462,0.555556,0.461538,NaN,NaN,NaN,NaN
34,Sunset,1:00:34,G2 Esports,MIBR,6,7,3,6,0,0,...,0.409091,0.590909,0.333333,0.461538,0.666667,0.538462,NaN,NaN,NaN,NaN
35,Split,1:08:34,G2 Esports,MIBR,5,5,7,7,3,1,...,0.535714,0.464286,0.466667,0.538462,0.333333,0.384615,NaN,NaN,NaN,NaN
36,Breeze,59:07,G2 Esports,MIBR,6,7,4,6,0,0,...,0.434783,0.565217,0.400000,0.461538,0.600000,0.538462,NaN,NaN,NaN,NaN


In [63]:
def make_predictions(df, predictors):
    training = df[df.index <= 163]
    validation = df[df.index > 163]
    rf.fit(training[predictors], training['Winner'])
    preds = rf.predict(validation[predictors])
    combined = pd.DataFrame(dict(actual=validation["Winner"], predicted=preds), index=validation.index)
    error = precision_score(validation["Winner"],preds)
    return combined, error

In [64]:
combined, error = make_predictions(df, predictors)

In [70]:
combined = combined.merge(df[["Team1", "Team2", "Map", "Winner"]], left_index=True, right_index=True)

In [71]:
combined.head(10)

,actual,predicted,Team 1 Code,Team 2 Code,Map Code,Winner_x,Team1,Team2,Map,Winner_y
164,0,0,5,7,7,0,Leviatán,100 Thieves,Haven,0
165,1,0,4,2,7,1,NRG Esports,G2 Esports,Haven,1
166,1,1,4,2,3,1,NRG Esports,G2 Esports,Bind,1
167,0,0,0,1,7,0,Sentinels,Cloud9,Haven,0
168,0,1,0,1,0,0,Sentinels,Cloud9,Split,0
169,1,1,10,5,1,1,Evil Geniuses,Leviatán,Icebox,1
170,1,1,10,5,2,1,Evil Geniuses,Leviatán,Sunset,1
171,0,0,6,9,7,0,LOUD,FURIA,Haven,0
172,1,0,6,9,1,1,LOUD,FURIA,Icebox,1
173,1,0,6,9,3,1,LOUD,FURIA,Bind,1


In [66]:
error

0.9166666666666666

In [76]:
 upcoming = [{'Map Code': 2 , 'Team 1 Code':0, 'Team 2 Code':7},{'Map Code': 1 , 'Team 1 Code':0, 'Team 2 Code':7},{'Map Code': 7 , 'Team 1 Code':0, 'Team 2 Code':7}]